# Mental Health in Tech Workplaces
Project for "Programming for Data Science"

COSE DA FARE / CHE MI VENGONO IN MENTE
- convertre tutto a Maiusc + Min nell'head
- raw source e piccola intro
- quando avrai dati sistemati, source anche di quello

In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st

## Exploration and cleaning of the dataset

### The dataset

In [160]:
mental_health_df = pd.read_csv("survey.csv")

### Useful informations

- In this dataset there are 1259 rows and 27 columnes (attributes)

- The age column has integer values

- All the other attribitues have object values

In [161]:
mental_health_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-null   object
 13  care_options               1259 non-null   object
 14  wellness

### Meaning of the columns


The first 4 columns (2 to 5) concern general informations about the individuals.

-  **Timestamp**: contains date, month, year and time

- **Age**

- **Gender** 

- **Country** 

- **state**: state relative to the USA

- **self_employed**: is the interviewee  self-employed?

- **family_History**: does the interviewee have a family history in mental illness?

- **treatment**: does the interviewee searched for a treatment?

- **work_interfere**: if the individual has a mental health condition, does he think that the latter interferes with its work?

- **no_employees**: number of employees of the interviewee's company
- **remote_work**: does the interviewee work remotly at least 50% of the time?

- **tech_company**: is the interviewee's employer primarily a technology company/organization?

- **benefits**: does the interviewee's employer provide mental health benefits to his employees

- **care_options**: does the interviewee know about the options for mental health care (offered by the employer)?

- **wellness_program**: has the interviewee's employer ever discussed mental health as part of an employee wellness program?

- **seek_help**: does the interviewee's employer provide resources to learn more about mental health issues and how to seek help?

- **anonymity**: is the anonymity of the interviewee protected if he chooses to take advantage of mental health or substance abuse treatment resources?

- **leave**: how easy is it for the interviewee to take medical leave for a mental health condition?

- **mental_health_consequence**: does the interviewee think that discussing a mental health issue with his employer would have negative consequences?

- **phys_health_consequence**: does the interviewee think that discussing a physical health issue with his employer would have negative consequences?

- **coworkers**: would the interviewee be willing to discuss a mental health issue with his coworkers?

- **supervisor**: would the interviewee be willing to discuss a mental health issue with his direct supervisor(s)?

- **mental_health_interview**: would the interviewee bring up a mental health issue with a potential employer in an interview?

- **phys_health_interview**: would the interviewee bring up a physical health issue with a potential employer in an interview? 

- **mental_vs_physical**: does tne interviewee feels that his employer takes mental health as seriously as physical health?

- **obs_consequence**: has the interviewee heard of or observed negative consequences for coworkers with mental health conditions in his workplace?

- **comments**: any additional notes or comments   


In [162]:
mental_health_df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


Here i converting the column's name to lowercase characters. In this way, manipulating the columns becomes easier.

In [163]:
mental_health_df.columns = mental_health_df.columns.map(str.lower)

### Dropping columns   

- Dropping the *timestamp* column because it isn't useful for the survey 

- Check how many null values in each columns

- Dropping the *comments* column because the are 1095 null values out of 1259 entries 



In [164]:
mental_health_df.isnull().sum()

timestamp                       0
age                             0
gender                          0
country                         0
state                         515
self_employed                  18
family_history                  0
treatment                       0
work_interfere                264
no_employees                    0
remote_work                     0
tech_company                    0
benefits                        0
care_options                    0
wellness_program                0
seek_help                       0
anonymity                       0
leave                           0
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical              0
obs_consequence                 0
comments                     1095
dtype: int64

In [165]:
mental_health_df.drop(columns=['timestamp', 'comments'], inplace = True)

- In the *state* column the are too many null values, since there are only USA's stat Even if we focus only on the USA, there are too few people for many states. Thus it would be misleading to say that a certain state is prone to cases of mental illness.

- For these reasons, the column has to be dropped.

In [166]:
mental_health_df['state'].value_counts()

CA    138
WA     70
NY     57
TN     45
TX     44
OH     30
IL     29
OR     29
PA     29
IN     27
MI     22
MN     21
MA     20
FL     15
NC     14
VA     14
WI     12
GA     12
MO     12
UT     11
CO      9
MD      8
AL      8
AZ      7
OK      6
NJ      6
KY      5
SC      5
IA      4
CT      4
DC      4
NV      3
VT      3
SD      3
KS      3
NH      3
WY      2
NM      2
NE      2
WV      1
ID      1
MS      1
RI      1
LA      1
ME      1
Name: state, dtype: int64

Here I want to highlight some important elements about the *countries* column 
- Since almost the 60% percent of the people come from USA (and the 5% from Canada), making certain assumptions about the data by referring to the country of origin could be risky. As in the previus case, we can't say that a country has more case of mental ilness, because the numbers of respondents across countries aren't uniform. 

- I filtered the European countries, in order to have a comparison with the USA. 

- Therfore, selecting only European countries, we can see that the majority of the individuals come from UK. Considering all the other countries (both european and non), there are many that have only one exponent. As before, this variety could be misleading. 

However, i will keep this column as additional information that could be useful.

In [167]:
mental_health_df['country'].value_counts()

United States             751
United Kingdom            185
Canada                     72
Germany                    45
Ireland                    27
Netherlands                27
Australia                  21
France                     13
India                      10
New Zealand                 8
Poland                      7
Switzerland                 7
Sweden                      7
Italy                       7
South Africa                6
Belgium                     6
Brazil                      6
Israel                      5
Singapore                   4
Bulgaria                    4
Austria                     3
Finland                     3
Mexico                      3
Russia                      3
Denmark                     2
Greece                      2
Colombia                    2
Croatia                     2
Portugal                    2
Moldova                     1
Georgia                     1
Bahamas, The                1
China                       1
Thailand  

In [168]:
eu_states = pd.read_csv('europe_states.csv')

l_eu = list()
count_eu = 0
count_nothing = 0
cont_usa = 0
cont_canada = 0
for i in range(len(mental_health_df)):
    if str(mental_health_df.iloc[i]['country']) in list(eu_states['name']):
        count_eu+=1
        l_eu.append(mental_health_df.iloc[i]['country'])
    elif str(mental_health_df.iloc[i]['country']) == 'United States':
        cont_usa+=1
    elif str(mental_health_df.iloc[i]['country']) == 'Canada':
        cont_canada +=1
    else:
        count_nothing+=1


print('People from Europe:',count_eu, '\nPeople from USA: ', cont_usa, '\nPeople from Canada: ',cont_canada, '\nPeople from other countries: ',count_nothing)

People from Europe: 358 
People from USA:  751 
People from Canada:  72 
People from other countries:  78


In [169]:
eu_serie = pd.Series(l_eu)
eu_serie.value_counts()

United Kingdom            185
Germany                    45
Netherlands                27
Ireland                    27
France                     13
Sweden                      7
Switzerland                 7
Poland                      7
Italy                       7
Belgium                     6
Bulgaria                    4
Finland                     3
Austria                     3
Croatia                     2
Greece                      2
Denmark                     2
Portugal                    2
Moldova                     1
Norway                      1
Latvia                      1
Hungary                     1
Bosnia and Herzegovina      1
Spain                       1
Romania                     1
Slovenia                    1
Czech Republic              1
dtype: int64

In [170]:
mental_health_df.drop(columns=['state'], inplace = True)

### Data cleaning 

In the *gender* column there are:
- many values that can be grouped under a single label

- non sense values 

- wrong spelled words

In order to clean this attribute, i will group under 'Female':
- female

- cis female that is, those women who are comfortable with their biological gender

- all words that refer to this genre

The same for the 'Male' gender.

For the other genders it's more complicated because the responses aren't clear and the labels are many. For instance, there is difference between transgender and transexual, so the label 'trans' could be incomplete. 

Thus i decided to group all this different cases under the label 'Other'. I understand that could seem hasty, but it's easier for the next analyses.


In [193]:
print('Unique values for the age column:\n',mental_health_df['gender'].unique())
print('\n\nUnique values for the age column:\n', mental_health_df['age'].unique())

Unique values for the age column:
 ['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah'
 'All' 'Enby' 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender'
 'cis-female/femme' 'Guy (-ish) ^_^' 'male leaning androgynous' 'Male '
 'Man' 'Trans woman' 'msle' 'Neuter' 'Female (trans)' 'queer'
 'Female (cis)' 'Mail' 'cis male' 'A little about you' 'Malr' 'p' 'femail'
 'Cis Man' 'ostensibly male, unsure what that really means']

Unique values for the age column:
 [         37          44          32          31          33          35
          39          42          23          29          36          27
          46          41          34          30          40          38
          50          24          18          28          26          22
          19          25          45          21         -29          43
      

In [198]:
female_list = ['Female', 'female', 'Cis Female', 'F' , 'Woman', 'f', 'Femake' , 'woman', 'Female ', 'cis-female/femme', 'Female (cis)', 'femail' ]
male_list = ['M','Male','male', 'm' ,'Male-ish', 'maile', 'Cis Male' , 'Mal', 'Male (CIS)', 'Make', 'Male ', 'Man','msle','Mail' , 'cis male', 'Malr', 'Cis Man' ]
other_list = ['Trans-female', 'something kinda male?', 'queer/she/they', 'non-binary', 'Nah', 'All','Enby', 'fluid','Genderqueer', 'Androgyne','Agender', 'Guy (-ish) ^_^','male leaning androgynous', 'Trans woman', 'Neuter','Female (trans)','queer', 'A little about you', 'p', 'ostensibly male, unsure what that really means']

mental_health_df.replace(female_list, 'Female', inplace=True)
mental_health_df.replace(male_list, 'Male', inplace=True)
mental_health_df.replace(other_list, 'Other', inplace=True)


In [200]:
# Cleaned column
mental_health_df['gender'].unique()

array(['Female', 'Male', 'Other'], dtype=object)

- checking quickly that there aren't wrong values in the other columns

In [187]:
for column in mental_health_df.iloc[:, 3:]:
    print(column, mental_health_df[column].unique())


self_employed [nan 'Yes' 'No']
family_history ['No' 'Yes']
treatment ['Yes' 'No']
work_interfere ['Often' 'Rarely' 'Never' 'Sometimes' nan]
no_employees ['6-25' 'More than 1000' '26-100' '100-500' '1-5' '500-1000']
remote_work ['No' 'Yes']
tech_company ['Yes' 'No']
benefits ['Yes' "Don't know" 'No']
care_options ['Not sure' 'No' 'Yes']
wellness_program ['No' "Don't know" 'Yes']
seek_help ['Yes' "Don't know" 'No']
anonymity ['Yes' "Don't know" 'No']
leave ['Somewhat easy' "Don't know" 'Somewhat difficult' 'Very difficult'
 'Very easy']
mental_health_consequence ['No' 'Maybe' 'Yes']
phys_health_consequence ['No' 'Yes' 'Maybe']
coworkers ['Some of them' 'No' 'Yes']
supervisor ['Yes' 'No' 'Some of them']
mental_health_interview ['No' 'Yes' 'Maybe']
phys_health_interview ['Maybe' 'No' 'Yes']
mental_vs_physical ['Yes' "Don't know" 'No']
obs_consequence ['No' 'Yes']
